In [74]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [68]:
import pandas as pd
from sklearn.preprocessing import Imputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import scipy.stats as ss
import itertools
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.manifold import TSNE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import LeaveOneOut
import mifs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import scipy
import numpy as np, scipy.stats as st
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from scipy import interp
from collections import Counter
import mifs

In [34]:
#importing data and processing labels for further analysis
gravier=pd.read_csv('gravier_inputs.csv',header=None)
gravier_labels=pd.read_csv('gravier_outputs.csv')
gravier_labels['V1']=gravier_labels['V1'].astype('category').cat.codes
gravier_labels['V1'].value_counts()

0    111
1     57
Name: V1, dtype: int64

## Selecting features using L1 regularized logistic regression

In [84]:
def LASSO(data,labels):
    X_train, X_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.3,
    random_state=0,stratify=labels)
    scaler = StandardScaler()
    scaler.fit(X_train.fillna(0))
    sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
    sel_.fit(scaler.transform(X_train.fillna(0)), y_train)
    selected_feat = X_train.columns[(sel_.get_support())]
    print('total features: {}'.format((X_train.shape[1])))
    print('selected features: {}'.format(len(selected_feat)))
    print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))
    X_train_reduced=X_train[selected_feat]
    print(X_train_reduced.shape)
    sm = SMOTE(random_state=2)
    X_train_sm, y_train_sm = sm.fit_sample(X_train_reduced, y_train)
    SVM=SVC(kernel="linear")
    tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    clf_svm = GridSearchCV(SVC(), tuned_parameters, cv=10)
    best_model_svm=clf_svm.fit(X_train_sm,y_train_sm)
    svm_train=SVC(kernel='linear',C=best_model_svm.best_params_['C'])
    svm_train.fit(X_train_sm,y_train_sm)
    print("SVM")
    print(classification_report(y_test,svm_train.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,svm_train.predict(X_test[selected_feat])))
    print("Random forest")
    rf=RandomForestClassifier(n_estimators=500)
    rf.fit(X_train_sm,y_train_sm)
    print(classification_report(y_test,rf.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,rf.predict(X_test[selected_feat])))
    print("Decision trees")
    dt=DecisionTreeClassifier()
    dt.fit(X_train_sm,y_train_sm)
    print(classification_report(y_test,dt.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,rf.predict(X_test[selected_feat])))

## Selecting feature using recursive feature elimination

In [82]:
def RFECV_analysis(data,labels):
    X_train, X_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.3,
    random_state=0,stratify=labels)
    selector = RFECV(RandomForestClassifier(), step=1, cv=3)
    selector.fit(X_train, y_train)
    selected_feat=X_train.columns[selector.support_==True]
    X_train_reduced=X_train[selected_feat]
    
    X_train_reduced=X_train[selected_feat]
    print(X_train_reduced.shape)
    sm = SMOTE(random_state=2)
    X_train_sm, y_train_sm = sm.fit_sample(X_train_reduced, y_train)
    SVM=SVC(kernel="linear")
    tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    clf_svm = GridSearchCV(SVC(), tuned_parameters, cv=10)
    best_model_svm=clf_svm.fit(X_train_sm,y_train_sm)
    svm_train=SVC(kernel='linear',C=best_model_svm.best_params_['C'])
    svm_train.fit(X_train_sm,y_train_sm)
    print("SVM")
    print(classification_report(y_test,svm_train.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,svm_train.predict(X_test[selected_feat])))
    print("Random forest")
    rf=RandomForestClassifier(n_estimators=500)
    rf.fit(X_train_sm,y_train_sm)
    print(classification_report(y_test,rf.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,rf.predict(X_test[selected_feat])))
    print("Decision trees")
    dt=DecisionTreeClassifier()
    dt.fit(X_train_sm,y_train_sm)
    print(classification_report(y_test,dt.predict(X_test[selected_feat])))
    print(X_test[selected_feat].shape)
    print(roc_auc_score(y_test,rf.predict(X_test[selected_feat])))




In [83]:
RFECV_analysis(gravier,gravier_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(117, 1996)


/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


SVM
              precision    recall  f1-score   support

           0       0.75      0.88      0.81        34
           1       0.64      0.41      0.50        17

    accuracy                           0.73        51
   macro avg       0.69      0.65      0.66        51
weighted avg       0.71      0.73      0.71        51

(51, 1996)
0.6470588235294117
Random forest
              precision    recall  f1-score   support

           0       0.80      0.82      0.81        34
           1       0.62      0.59      0.61        17

    accuracy                           0.75        51
   macro avg       0.71      0.71      0.71        51
weighted avg       0.74      0.75      0.74        51

(51, 1996)
0.7058823529411765
Decision trees
              precision    recall  f1-score   support

           0       0.76      0.74      0.75        34
           1       0.50      0.53      0.51        17

    accuracy                           0.67        51
   macro avg       0.63      0.63  

In [85]:
LASSO(gravier,gravier_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


total features: 2905
selected features: 63
features with coefficients shrank to zero: 2842
(117, 63)
SVM
              precision    recall  f1-score   support

           0       0.79      0.76      0.78        34
           1       0.56      0.59      0.57        17

    accuracy                           0.71        51
   macro avg       0.67      0.68      0.67        51
weighted avg       0.71      0.71      0.71        51

(51, 63)
0.6764705882352942
Random forest
              precision    recall  f1-score   support

           0       0.86      0.88      0.87        34
           1       0.75      0.71      0.73        17

    accuracy                           0.82        51
   macro avg       0.80      0.79      0.80        51
weighted avg       0.82      0.82      0.82        51

(51, 63)
0.7941176470588236
Decision trees
              precision    recall  f1-score   support

           0       0.78      0.85      0.82        34
           1       0.64      0.53      0.58    

In [78]:
#importing data and processing labels for further analysis
gordon=pd.read_csv('gordon_inputs.csv',header=None)
gordon_labels=pd.read_csv('gordon_outputs.csv')
gordon_labels['V1']=gordon_labels['V1'].astype('category').cat.codes
gordon_labels['V1'].value_counts()

0    150
1     31
Name: V1, dtype: int64

In [86]:
RFECV_analysis(gordon,gordon_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(126, 599)


/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        46
           1       0.89      0.89      0.89         9

    accuracy                           0.96        55
   macro avg       0.93      0.93      0.93        55
weighted avg       0.96      0.96      0.96        55

(55, 599)
0.9335748792270532
Random forest
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        46
           1       1.00      0.89      0.94         9

    accuracy                           0.98        55
   macro avg       0.99      0.94      0.97        55
weighted avg       0.98      0.98      0.98        55

(55, 599)
0.9444444444444444
Decision trees
              precision    recall  f1-score   support

           0       0.94      0.96      0.95        46
           1       0.75      0.67      0.71         9

    accuracy                           0.91        55
   macro avg       0.84      0.81    

In [87]:
LASSO(gordon,gordon_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


total features: 12533
selected features: 34
features with coefficients shrank to zero: 12499
(126, 34)
SVM
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        46
           1       1.00      0.89      0.94         9

    accuracy                           0.98        55
   macro avg       0.99      0.94      0.97        55
weighted avg       0.98      0.98      0.98        55

(55, 34)
0.9444444444444444
Random forest
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        46
           1       0.89      0.89      0.89         9

    accuracy                           0.96        55
   macro avg       0.93      0.93      0.93        55
weighted avg       0.96      0.96      0.96        55

(55, 34)
0.9335748792270532
Decision trees
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        46
           1       1.00      0.89      0.94  

In [89]:
#importing data and processing labels for further analysis
golub=pd.read_csv('golub_inputs.csv',header=None)
golub_labels=pd.read_csv('golub_outputs.csv')
golub_labels['V1']=golub_labels['V1'].astype('category').cat.codes
golub_labels['V1'].value_counts()

0    47
1    25
Name: V1, dtype: int64

In [90]:
RFECV_analysis(golub,golub_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(50, 4021)


/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


SVM
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       1.00      0.88      0.93         8

    accuracy                           0.95        22
   macro avg       0.97      0.94      0.95        22
weighted avg       0.96      0.95      0.95        22

(22, 4021)
0.9375
Random forest
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       1.00      0.88      0.93         8

    accuracy                           0.95        22
   macro avg       0.97      0.94      0.95        22
weighted avg       0.96      0.95      0.95        22

(22, 4021)
0.9375
Decision trees
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        14
           1       0.86      0.75      0.80         8

    accuracy                           0.86        22
   macro avg       0.86      0.84      0.85        22
weigh

In [91]:
LASSO(golub,golub_labels)

total features: 7129
selected features: 25
features with coefficients shrank to zero: 7104
(50, 25)


/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM
              precision    recall  f1-score   support

           0       0.82      1.00      0.90        14
           1       1.00      0.62      0.77         8

    accuracy                           0.86        22
   macro avg       0.91      0.81      0.84        22
weighted avg       0.89      0.86      0.85        22

(22, 25)
0.8125
Random forest
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       1.00      0.88      0.93         8

    accuracy                           0.95        22
   macro avg       0.97      0.94      0.95        22
weighted avg       0.96      0.95      0.95        22

(22, 25)
0.9375
Decision trees
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.88      0.88      0.88         8

    accuracy                           0.91        22
   macro avg       0.90      0.90      0.90        22
weighted 

In [92]:
#importing data and processing labels for further analysis
alon=pd.read_csv('alon_inputs.csv',header=None)
alon_labels=pd.read_csv('alon_outputs.csv')
alon_labels['V1']=alon_labels['V1'].astype('category').cat.codes
alon_labels['V1'].value_counts()

1    40
0    22
Name: V1, dtype: int64

In [94]:
LASSO(alon,alon_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


total features: 2000
selected features: 23
features with coefficients shrank to zero: 1977
(43, 23)
SVM
              precision    recall  f1-score   support

           0       0.83      0.71      0.77         7
           1       0.85      0.92      0.88        12

    accuracy                           0.84        19
   macro avg       0.84      0.82      0.82        19
weighted avg       0.84      0.84      0.84        19

(19, 23)
0.8154761904761905
Random forest
              precision    recall  f1-score   support

           0       0.78      1.00      0.88         7
           1       1.00      0.83      0.91        12

    accuracy                           0.89        19
   macro avg       0.89      0.92      0.89        19
weighted avg       0.92      0.89      0.90        19

(19, 23)
0.9166666666666667
Decision trees
              precision    recall  f1-score   support

           0       0.88      1.00      0.93         7
           1       1.00      0.92      0.96     

In [95]:
RFECV_analysis(alon,alon_labels)

/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(43, 286)
SVM
              precision    recall  f1-score   support

           0       0.60      0.43      0.50         7
           1       0.71      0.83      0.77        12

    accuracy                           0.68        19
   macro avg       0.66      0.63      0.63        19
weighted avg       0.67      0.68      0.67        19

(19, 286)
0.6309523809523809
Random forest


/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/shayantan/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


              precision    recall  f1-score   support

           0       0.86      0.86      0.86         7
           1       0.92      0.92      0.92        12

    accuracy                           0.89        19
   macro avg       0.89      0.89      0.89        19
weighted avg       0.89      0.89      0.89        19

(19, 286)
0.8869047619047619
Decision trees
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         7
           1       1.00      0.42      0.59        12

    accuracy                           0.63        19
   macro avg       0.75      0.71      0.63        19
weighted avg       0.82      0.63      0.62        19

(19, 286)
0.8869047619047619
